# PROBLEMA 3: **Cáncer de Seno**  

### Entrega: Daniel Rojo Mata

## **Enunciado del problema**

Utiliza el conjunto de datos de cáncer de seno de Wisconsin, que contiene 699 registros de tumores, de los cuales 458 son benignos y 241 son malignos. Cada registro consta de los siguientes atributos:

### **1. Preparación de los datos:**
   - Divide aleatoriamente el conjunto de datos en un subconjunto de entrenamiento con el 60 % de los datos, un subconjunto de validación con el 20 % y un subconjunto de prueba con el 20 % restante, utilizando 0 como semilla para tu generador de números aleatorios.

### **2. Entrenamiento del clasificador:**
   - Entrena distintos clasificadores para los tumores de seno y evalúalos tanto con el subconjunto de entrenamiento como con el subconjunto de validación, discutiendo su desempeño.

### **3. Manejo de datos faltantes:**
   - Investiga estrategias para rellenar los datos faltantes, ya que existen 16 registros en el conjunto de datos con un atributo no especificado. Utiliza las estrategias que consideres más adecuadas y discute el impacto en el desempeño del clasificador.

### **4. Evaluación del modelo:**
   - Reporta el porcentaje de predicciones correctas en el subconjunto de prueba para el clasificador que tenga el mejor rendimiento en el subconjunto de validación.

## **Descripción del conjunto de datos**
El conjunto de datos de cáncer de seno de Wisconsin incluye registros de tumores con las siguientes características:
- **Atributos:** Contiene varias mediciones y características relacionadas con los tumores.
- **Etiquetas:** Indica si el tumor es benigno o maligno.


# **SOLUCIÓN**

## Librerías

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

## Se procesa la data

In [2]:
archivo = "cancer.csv" # Nombre del archivo, es importante que esté en la misma carpeta que este archivo
data = pd.read_csv(archivo, # Se importa el archivo
                  header=None, # Quitamos el encabezado
                  sep=',' # Separador
                  )

## Observación: Sobre el procesamiento de la data 
Los datos están organizados de la siguiente manera:

In [ ]:
data.head(5) # Ver los primeros 5 elementos de la data

Tomando el primer elemento se obtiene: 

In [ ]:
data.iloc[0,:] # Ver el primer renglón

Por lo que no es posible procesar de una manera adecuada los datos. 
Es por eso que se usa la función **procesar_data** la cual retorna 
los datos como una lista de listas, en donde cada lista contiene elementos de tipo entero.

In [3]:
def procesar_data(data):
    """
    Función que retorna una lista de listas con valores enteros, 
    convirtiendo los datos del DataFrame en una estructura manejable.

    Parámetros: 
        data: DataFrame de pandas con los datos en formato de una sola columna con valores separados por comas.

    Return: 
        lista_numerica: lista de listas con valores enteros, manteniendo los "?" como están.
    """
    
    lista_inicial = data.values.tolist()  # Convierte el DataFrame en una lista de listas (cada fila como lista).
    lista_numerica = []  # Lista que almacenará los datos convertidos.

    for lista in lista_inicial:  # Itera sobre cada fila del dataset.
        for i in range(len(lista)):  
            lista_split = lista[i].split(",")  # Divide la cadena en una lista separando por comas.
            lista_auxiliar = []  # Lista temporal para almacenar los valores convertidos.

            for k in lista_split:  # Itera sobre los valores de la lista dividida.
                if k != "?":  # Si el valor no es un "?", lo convierte en entero.
                    lista_auxiliar.append(int(k))
                else:  # Si es "?", lo mantiene como está para su posterior tratamiento.
                    lista_auxiliar.append(k)

            lista_numerica.append(lista_auxiliar)  # Agrega la lista procesada a la lista principal.

    return lista_numerica  # Retorna la lista de listas con valores convertidos.

# Lista inicial

Lista que se usará de aquí en adelante. Ejecutar la celda.

In [5]:
lista_inicial = procesar_data(data) # Se procesa la data con la función anterior

# Gráficas de la data 

In [6]:
data_numpy = np.array(lista_inicial)  # Convertimos la lista en un array NumPy
X = data_numpy[:, 1:-1]  # Excluimos la primera columna (ID) y la última (Clase)
y = data_numpy[:, -1]  # Última columna es la clase (2 = benigno, 4 = maligno)

In [ ]:
plt.figure(figsize=(12, 10))
columnas = ['Grosor tumor', 'Uniformidad tamaño célula', 'Uniformidad forma célula',
            'Adhesión marginal', 'Tamaño célula epitelial', 'Núcleos desnudos',
            'Cromatina blanda', 'Nucléolos normales', 'Mitosis']

for i in range(len(columnas)):
    plt.subplot(3, 3, i+1)
    sns.histplot(X[:, i], bins=10, kde=True)
    plt.xlabel(columnas[i])
    plt.ylabel('Frecuencia')
    plt.title(f'Histograma de {columnas[i]}')

plt.tight_layout()
plt.show()

# Datos faltantes

Se tienen 16 registros en los cuales no se tiene completa la información.
La función **listas_con_interrogacion** devuelve estas listas (registros).

In [8]:
def listas_con_interrogacion(lista_de_listas):
    """
    Función que devuelve las listas que incluyen el carácter "?".
    
    Parámetros: 
        lista_de_listas: lista cuyos elementos son listas con valores numéricos
        o con el carácter "?".
    
    Return:
        Una lista que contiene solo las sublistas que contienen el carácter "?".
    """
    # Utiliza una comprensión de listas para filtrar las sublistas que contienen "?".
    return [lista for lista in lista_de_listas if "?" in lista]

In [9]:
# Listas en donde se tiene un elemento con "?"
listas_interrogacion = listas_con_interrogacion(lista_inicial) 
# Son 16 cosos
listas_interrogacion

[[1057013, 8, 4, 5, 1, 2, '?', 7, 3, 1, 4],
 [1096800, 6, 6, 6, 9, 6, '?', 7, 8, 1, 2],
 [1183246, 1, 1, 1, 1, 1, '?', 2, 1, 1, 2],
 [1184840, 1, 1, 3, 1, 2, '?', 2, 1, 1, 2],
 [1193683, 1, 1, 2, 1, 3, '?', 1, 1, 1, 2],
 [1197510, 5, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [1241232, 3, 1, 4, 1, 2, '?', 3, 1, 1, 2],
 [169356, 3, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [432809, 3, 1, 3, 1, 2, '?', 2, 1, 1, 2],
 [563649, 8, 8, 8, 1, 2, '?', 6, 10, 1, 4],
 [606140, 1, 1, 1, 1, 2, '?', 2, 1, 1, 2],
 [61634, 5, 4, 3, 1, 2, '?', 2, 3, 1, 2],
 [704168, 4, 6, 5, 6, 7, '?', 4, 9, 1, 2],
 [733639, 3, 1, 1, 1, 2, '?', 3, 1, 1, 2],
 [1238464, 1, 1, 1, 1, 1, '?', 2, 1, 1, 2],
 [1057067, 1, 1, 1, 1, 1, '?', 1, 1, 1, 2]]

Por lo que antes de dividir el conjunto de datos en las proporciones que se piden,
se piensan las siguientes estrategias para "completar" esos datos.

# **Estrategias para llenar los datos faltantes**

En este trabajo se emplean dos estrategias para manejar los valores faltantes representados por `"?"`:  

## **1. Eliminar filas con `?`**  
Esta estrategia consiste en eliminar directamente todas las filas que contienen valores desconocidos.  

✅ **Ventajas:**  
- Simple de implementar.  
- Evita cualquier sesgo introducido por valores imputados.  

❌ **Desventajas:**  
- Se pierde información útil si hay muchas filas afectadas.  
- Puede reducir el tamaño del conjunto de datos, lo que impacta la calidad del modelo.  

## **2. Imputación con la mediana**  
En este enfoque, los valores faltantes se reemplazan con la mediana de la columna correspondiente. La mediana es robusta a valores extremos, lo que la hace adecuada para datos sesgados.  

✅ **Ventajas:**  
- Conserva la mayor cantidad posible de datos.  
- Reduce el impacto de valores atípicos en comparación con la media.  

❌ **Desventajas:**  
- Puede introducir sesgo si los datos faltantes no están distribuidos aleatoriamente.  
- No captura variabilidad en los valores reemplazados.  

## Función para cálculo de precisión

In [10]:
# Función para calcular precisión y retornar los valores
def calcular_precision(modelo, X_ent, y_ent, X_val, y_val, X_test, y_test):
    predicciones_ent = modelo.predict(X_ent)
    predicciones_val = modelo.predict(X_val)
    predicciones_test = modelo.predict(X_test)

    accuracy_ent = accuracy_score(y_ent, predicciones_ent) * 100
    accuracy_val = accuracy_score(y_val, predicciones_val) * 100
    accuracy_test = accuracy_score(y_test, predicciones_test) * 100

    # Imprimir las precisiones
    print(f"Precisión en entrenamiento: {accuracy_ent:.2f}%")
    print(f"Precisión en validación: {accuracy_val:.2f}%")
    print(f"Precisión en prueba: {accuracy_test:.2f}%")

    # Regresar las precisiones
    return accuracy_ent, accuracy_val, accuracy_test

## **Estrategia 1**: Eliminar filas con "?"

Se borran estos datos en **lista_inicial**.

In [11]:
# Filtra las sublistas de lista_inicial para eliminar aquellas que contienen el carácter "?".
data_numpy_filtrado = [sublista for sublista in lista_inicial if sublista not in listas_con_interrogacion(lista_inicial)]

In [12]:
# Se observa que las lingitudes coinciden
len(data_numpy_filtrado) == len(lista_inicial)-len(listas_interrogacion)

True

In [13]:
# 699 - 16 = 683
len(data_numpy_filtrado)

683

## División del conjunto de datos utilizando las filas eliminadas con "?"

Para dividir la data, se usan los datos en **data_numpy_filtrado** generado celdas arriba

In [14]:
# Convertir 'data_numpy_filtrado' en un array para usar [:, :]
X_interrogacion = np.array(data_numpy_filtrado)[:, :-1]  # Todas las columnas excepto la última
y_interrogacion = np.array(data_numpy_filtrado)[:, -1]   # Solo la última columna

# Dividir en conjunto de entrenamiento (60%) y el resto (40%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X_interrogacion, y_interrogacion, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

### Clasificador Gaussiano y Clasificador Multinomial importados de Sklearn 

In [15]:
print("-"*50)

# Ver las dimensiones de los conjuntos anteriores
print(f'Tamaño del conjunto de entrenamiento: {X_ent.shape}')
print(f'Tamaño del conjunto de validación: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')

print("-"*50)

# Crear y entrenar el clasificador Naive Bayes Gaussiano
nb_classifier = GaussianNB()
nb_classifier.fit(X_ent, y_ent)
print("Resultados para Naive Bayes Gaussiano:\n")
accuracy_ent_interrogacion_gaussiano, accuracy_val_interrogacion_gaussiano, accuracy_test_interrogacion_gaussiano = calcular_precision(nb_classifier, X_ent, y_ent, X_val, y_val, X_test, y_test)

print("-"*50)

# Crear y entrenar el clasificador Naive Bayes Multinomial
miltinomial = MultinomialNB()
miltinomial.fit(X_ent, y_ent)
print("Resultados para Naive Bayes Multinomial:\n")
accuracy_ent_interrogacion_multinomial, accuracy_val_interrogacion_multinomial, accuracy_test_interrogacion_multinomial = calcular_precision(miltinomial, X_ent, y_ent, X_val, y_val, X_test, y_test)

--------------------------------------------------
Tamaño del conjunto de entrenamiento: (409, 10)
Tamaño del conjunto de validación: (137, 10)
Tamaño del conjunto de prueba: (137, 10)
--------------------------------------------------
Resultados para Naive Bayes Gaussiano:

Precisión en entrenamiento: 66.75%
Precisión en validación: 70.80%
Precisión en prueba: 70.80%
--------------------------------------------------
Resultados para Naive Bayes Multinomial:

Precisión en entrenamiento: 96.82%
Precisión en validación: 89.05%
Precisión en prueba: 93.43%


## **Estrategia 2**: imputación con la mediana



In [16]:
# Se convierte la lista_inicial en un array de numpy
lista_inicial_array = np.array(lista_inicial, dtype=object)  # dtype=object porque hay strings

# Reemplazar "?" con NaN
lista_inicial_array[lista_inicial_array == "?"] = np.nan

# Convertir a tipo numérico
lista_inicial_array = lista_inicial_array.astype(float)

# Calcular la mediana ignorando NaN
medianas = np.nanmedian(lista_inicial_array, axis=0)

# Reemplazar los NaN con la mediana de la columna correspondiente
indices_nan = np.where(np.isnan(lista_inicial_array))
lista_inicial_array[indices_nan] = np.take(medianas, indices_nan[1])

### División del conjunto de datos utilizando imputación con la mediana

Para dividir la data, se usan los datos en **lista_inicial_array** (celda de arriba) 

In [17]:
# Convertimos 'lista_inicial_array' a un array NumPy si aún no lo es
data_numpy_mediana = np.array(lista_inicial_array)  

# Separa características (X) y etiquetas (y)
X_mediana = data_numpy_mediana[:, :-1]  # Todas las columnas excepto la última
y_mediana = data_numpy_mediana[:, -1]   # Solo la última columna

# Convertir etiquetas a enteros por si están como flotantes
y_mediana = y_mediana.astype(int)

# Dividir en conjunto de entrenamiento (60%) y el resto (40%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X_mediana, y_mediana, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

### Clasificador Gaussiano y Clasificador Miltinomial importados de Sklearn 

In [23]:
print("-"*50)

# Ver las dimensiones de los conjuntos
print(f'Tamaño del conjunto de entrenamiento: {X_ent.shape}')
print(f'Tamaño del conjunto de validación: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')

print("-"*50)

# Crear y entrenar el clasificador Naive Bayes Gaussiano
nb_classifier = GaussianNB()
nb_classifier.fit(X_ent, y_ent)
print("Resultados para Naive Bayes Gaussiano:\n")
accuracy_ent_mediana_gaussiano, accuracy_val_mediana_gaussiano, accuracy_test_mediana = calcular_precision(nb_classifier, X_ent, y_ent, X_val, y_val, X_test, y_test)

print("-"*50)

# Crear y entrenar el clasificador Naive Bayes Multinomial
miltinomial = MultinomialNB()
miltinomial.fit(X_ent, y_ent)
print("Resultados para Naive Bayes Multinomial:\n")
accuracy_ent_mediana_multinomial, accuracy_val_mediana_multinomial, accuracy_test_mediana_multinomial = calcular_precision(miltinomial, X_ent, y_ent, X_val, y_val, X_test, y_test)

--------------------------------------------------
Tamaño del conjunto de entrenamiento: (419, 10)
Tamaño del conjunto de validación: (140, 10)
Tamaño del conjunto de prueba: (140, 10)
--------------------------------------------------
Resultados para Naive Bayes Gaussiano:

Precisión en entrenamiento: 82.82%
Precisión en validación: 80.00%
Precisión en prueba: 89.29%
--------------------------------------------------
Resultados para Naive Bayes Multinomial:

Precisión en entrenamiento: 94.75%
Precisión en validación: 91.43%
Precisión en prueba: 95.71%


## Comparación de las estrategias

In [27]:
# Analizar el desempeño para Naive Bayes Gaussiano y Naive Bayes Multinomial
print("\nAnálisis del desempeño eliminando la data con '?':")
print("-" * 50)
print("Desempeño con Naive Bayes Gaussiano:")
print(f"La precisión en entrenamiento es {accuracy_ent_interrogacion_gaussiano:.2f}%")
print(f"La precisión en validación es {accuracy_val_interrogacion_gaussiano:.2f}%")
print(f"La precisión en prueba es {accuracy_test_interrogacion_gaussiano:.2f}%")

print("\nDesempeño con Naive Bayes Multinomial:")
print(f"La precisión en entrenamiento es {accuracy_ent_interrogacion_multinomial:.2f}%")
print(f"La precisión en validación es {accuracy_val_interrogacion_multinomial:.2f}%")
print(f"La precisión en prueba es {accuracy_test_interrogacion_multinomial:.2f}")

print("\n")
print("\nAnálisis del desempeño utilizando la mediana:")
print("-" * 50)
print("Desempeño con Naive Bayes Gaussiano:")
print(f"La precisión en entrenamiento es {accuracy_ent_mediana_gaussiano:.2f}%")
print(f"La precisión en validación es {accuracy_val_mediana_gaussiano:.2f}%")
print(f"La precisión en prueba es {accuracy_test_mediana_gaussiano:.2f}%")

print("\nDesempeño con Naive Bayes Multinomial:")
print(f"La precisión en entrenamiento es {accuracy_ent_mediana_multinomial:.2f}%")
print(f"La precisión en validación es {accuracy_val_mediana_multinomial:.2f}%")
print(f"La precisión en prueba es {accuracy_test_mediana_multinomial:.2f}%")


Análisis del desempeño eliminando la data con '?':
--------------------------------------------------
Desempeño con Naive Bayes Gaussiano:
La precisión en entrenamiento es 66.75%
La precisión en validación es 70.80%
La precisión en prueba es 70.80%

Desempeño con Naive Bayes Multinomial:
La precisión en entrenamiento es 96.82%
La precisión en validación es 89.05%
La precisión en prueba es 93.43



Análisis del desempeño utilizando la mediana:
--------------------------------------------------
Desempeño con Naive Bayes Gaussiano:
La precisión en entrenamiento es 82.82%
La precisión en validación es 80.00%
La precisión en prueba es 89.29%

Desempeño con Naive Bayes Multinomial:
La precisión en entrenamiento es 94.75%
La precisión en validación es 91.43%
La precisión en prueba es 95.71%


# DISCUSIÓN

### **Análisis de estrategias para completar datos faltantes**

Se evaluaron dos estrategias para manejar los valores faltantes en el conjunto de datos:

1. **Imputación con la mediana:** Se reemplazaron los valores faltantes con la mediana de la columna correspondiente.  
2. **Eliminación de registros con valores faltantes:** Se eliminaron las filas que contenían valores desconocidos ("?").  

Los resultados en el conjunto de validación fueron los siguientes:

- **Accuracy con imputación de la mediana para el clasificador Gaussiano:** **80.00%**
- **Accuracy con imputación de la mediana para el clasificador Multinomial:** **91.43%**  
- **Accuracy eliminando registros con valores "?" y con clasificador Gaussiano:** **70.80%**
- **Accuracy eliminando registros con valores "?" y con clasificador Multinomial:** **89.05%**  

Estos resultados indican que la imputación con la mediana es una mejor estrategia en este caso, ya que permite conservar la mayor parte de los datos y mejora el rendimiento del modelo.

Aunque los registros con valores faltantes eran solo 16 de un total de 699 (menos del 5% del dataset), su eliminación impactó significativamente la precisión del modelo, sobre todo al hacer uso del clasificador Gaussiano. Inicialmente, se esperaba que la diferencia no fuera tan grande pues no es un porcentaje tan relevante, pero la caída en el desempeño sugiere que la pérdida de información sí fue importante al menos para este clasificador.

Además, los mismos efectos se observaron en los conjuntos de entrenamiento y prueba, reafirmando que la imputación con la mediana es la mejor opción entre las dos estrategias evaluadas destacando el hecho de que le mejor clasificador en ambos casos fue el *Multinomial*. 

Con base a lo obtenido, la mejor estrategia fue usar la *imputación de la mediana* con un clasificador *multinomial*.

---

### **¿Por qué usar Naive Bayes Gaussiano?**

Se optó por el uso de **Naive Bayes Gaussiano (GNB)** debido a las siguientes razones:

1. **Eficiente en datos numéricos**
GNB supone que los datos siguen una **distribución normal (gaussiana)** dentro de cada clase. Aunque nuestro análisis inicial de los datos (por medio de histogramas presentados en las primeras secciones) mostró que las variables **no siguen estrictamente una distribución normal**, en muchos casos **GNB sigue funcionando bien en la práctica**, especialmente con conjuntos de datos pequeños o medianos, como es el caso presentado.

2. **Simple y rápido**
Naive Bayes es un modelo **probabilístico interpretable y computacionalmente eficiente**. En comparación con modelos más complejos, **requiere menos datos para entrenar y es menos propenso al sobreajuste**, además de que **GNB es particularmente útil cuando el tamaño del dataset es moderado y se requiere un modelo rápido y explicable**.

3. **Independencia**
A pesar de que algunas variables pueden estar correlacionadas (por ejemplo, el tamaño de la célula epitelial y la uniformidad del tamaño de la célula), **Naive Bayes asume que todas las características son independientes**. En muchos problemas de clasificación médica, esta asunción resulta suficientemente útil, aunque no en todos los casos. 

---

### **¿Por qué usar Naive Bayes Multinomial?**

Se eligió el **Naive Bayes Multinomial (NBM)** debido a las siguientes razones:

1. **Datos de frecuencia o discretos:**  
   NBM es especialmente adecuado cuando las características representan **conteos o frecuencias** de eventos. Esto lo hace útil para problemas de clasificación donde los datos son numéricos, pero con una alta dispersión, como podría ser el caso que se presenta.

2. **Características dispersas:**  
   El **NBM** maneja bien las **características dispersas** y **no asume distribuciones normales** para los datos, lo que mejora su desempeño cuando las variables tienen alta variabilidad.

3. **Datos tabulares y texto:**  
   **NBM** funciona bien en **datos tabulares**, como el presente, donde las características pueden ser discretizadas y tratadas como frecuencias. 

# **CONCLUSIÓN**

A pesar de que los datos no siguen perfectamente una distribución normal, como lo demuestran los histogramas iniciales, Naive Bayes Gaussiano es una opción "razonable" debido a su eficiencia, interpretabilidad y buen desempeño en casos donde la cantidad de datos no es demasiado grande, como es el caso que se da. El desempeño, al menos utilizando la mediana, es un valor bastante representativo, con una cantidad del 80% de precisión en la validación. 

El análisis demuestra que, aunque el **Naive Bayes Gaussiano** proporciona buenos resultados, la **imputación con la mediana** y el **Naive Bayes Multinomial** proporcionan un rendimiento superior en términos de precisión, especialmente en el conjunto de validación. Esto sugiere que el **Naive Bayes Multinomial** podría ser la mejor opción en este problema, particularmente porque los datos presentan características dispersas.

Finalmente, la elección del modelo dependerá de varios factores, como la precisión requerida, la complejidad del modelo y de los datos. En este caso, ambos modelos de **Naive Bayes** ofrecen soluciones eficientes y fáciles de interpretar, pero con el **Multinomial** destacándose como la mejor opción para el conjunto de datos analizado.